In [ ]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier

class ACO:
    def __init__(self, num_ants, num_iterations, num_hidden_layers, hidden_layer_size, evaporation_rate, alpha, beta, Q, elitism_rate):
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.num_hidden_layers = num_hidden_layers
        self.hidden_layer_size = hidden_layer_size
        self.evaporation_rate = evaporation_rate
        self.alpha = alpha
        self.beta = beta
        self.Q = Q
        self.elitism_rate = elitism_rate

    def run(self, dataset):
        # Load data
        df = pd.read_csv(r'C:\Users\vijay\Downloads\dataset.csv')

        # Extract features and target variable
        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values

        # Initialize pheromone matrix
        num_weights = self.num_inputs * self.hidden_layer_size + \
                      (self.num_hidden_layers - 1) * self.hidden_layer_size**2 + \
                      self.hidden_layer_size
        pheromone = np.ones((num_weights, 2)) / num_weights

        # Initialize best solution and fitness
        best_solution = None
        best_fitness = -np.inf

        # Iterate for num_iterations
        for iteration in range(self.num_iterations):
            # Initialize ants' solutions and fitness
            solutions = np.zeros((self.num_ants, num_weights))
            fitness = np.zeros(self.num_ants)

            # Generate solutions for each ant
            for i in range(self.num_ants):
                solutions[i] = self.generate_solution(pheromone)
                fitness[i] = self.fitness(solutions[i], X, y)

            # Update best solution and fitness
            if np.max(fitness) > best_fitness:
                best_solution = solutions[np.argmax(fitness)]
                best_fitness = np.max(fitness)

            # Update pheromone
            pheromone = self.update_pheromone(pheromone, solutions, fitness)

        # Build neural network with best solution
        nn = self.create_nn(best_solution)
        nn.fit(X, y)

        return nn, best_solution, best_fitness

    def generate_solution(self, pheromone):
        # Initialize solution
        solution = np.zeros_like(pheromone)

        # Compute probabilities for each weight value
        probs = np.zeros_like(pheromone)
        for i in range(probs.shape[0]):
            for j in range(probs.shape[1]):
                probs[i, j] = pheromone[i, j]**self.alpha

        # Choose weight values for each weight
        for i in range(solution.shape[0]):
            for j in range(solution.shape[1]):
                if np.random.rand() < probs[i, 1] / np.sum(probs[i]):
                    solution[i, j] = 1

        return solution

    def fitness(self, solution, X, y):
        # Build neural network with given solution
        nn = self.create_nn(solution)
        nn.fit(X, y)

        # Compute accuracy score
        accuracy = nn.score(X, y)

        return accuracy

def update_pheromone(self, pheromone, solutions, fitness):
        # Rank solutions by fitness
        sorted_indices = np.argsort(fitness)[::-1]

        # Compute elite solutions
        elites_count = int(self.elitism_rate * self.num_ants)
        elites_indices = sorted_indices[:elites_count]

        # Update pheromone on elite trails
        for index in elites_indices:
            for i in range(self.num_cities - 1):
                city1 = solutions[index][i]
                city2 = solutions[index][i+1]
                pheromone[city1][city2] += self.elite_pheromone

        # Update pheromone on non-elite trails
        for index in sorted_indices[elites_count:]:
            for i in range(self.num_cities - 1):
                city1 = solutions[index][i]
                city2 = solutions[index][i+1]
                pheromone[city1][city2] += self.pheromone_increment
                pheromone[city2][city1] += self.pheromone_increment

        # Enforce pheromone limits
        np.clip(pheromone, self.pheromone_min, self.pheromone_max, out=pheromone)

        return pheromone 

